# TensorBoard

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/tensorflow/alex/mnist/input_data', one_hot=True)

Extracting /tmp/tensorflow/alex/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/alex/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/alex/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/alex/mnist/input_data/t10k-labels-idx1-ubyte.gz


In [3]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

### Graph

In [4]:
run_id = 'initial'
log_dir = '/home/jovyan/work/logs/alex/' + run_id

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, max_outputs=10)

with tf.name_scope('weights'):
    W = tf.Variable(initial_value=tf.zeros([784, 10], dtype=tf.float32), name='weights', trainable=True)
    variable_summaries(W)
with tf.name_scope('biases'):
    b = tf.Variable(initial_value=tf.zeros([10]), dtype=tf.float32, name='bias', trainable=True)
    variable_summaries(b)
with tf.name_scope('Wx_plus_b'):
    y = tf.matmul(x, W) + b
    tf.summary.histogram('predictions', y)

with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)

tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cross_entropy)


with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()

### Initialize

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test')

print(log_dir)

/home/jovyan/work/logs/alex/initial


### Train

In [6]:
for i in range(1000):
    if i % 10 == 0:  # Record summaries and test-set accuracy
        summary, acc = sess.run([merged, accuracy], feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' % (i, acc))
    else:  # Record train set summaries, and train
        xs, ys = mnist.train.next_batch(100)
        if i % 100 == 99:  # Record execution stats
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary, _ = sess.run([merged, train_step],
                              feed_dict={x: xs, y_: ys},
                              options=run_options,
                              run_metadata=run_metadata)
            train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
            train_writer.add_summary(summary, i)
            print('Adding run metadata for', i)
        else:  # Record a summary
            summary, _ = sess.run([merged, train_step], feed_dict={x: xs, y_: ys})
            train_writer.add_summary(summary, i)

train_writer.close()
test_writer.close()

Accuracy at step 0: 0.098
Accuracy at step 10: 0.719
Accuracy at step 20: 0.8312
Accuracy at step 30: 0.8688
Accuracy at step 40: 0.8703
Accuracy at step 50: 0.8703
Accuracy at step 60: 0.8666
Accuracy at step 70: 0.8828
Accuracy at step 80: 0.8787
Accuracy at step 90: 0.8911
Adding run metadata for 99
Accuracy at step 100: 0.8918
Accuracy at step 110: 0.8922
Accuracy at step 120: 0.8925
Accuracy at step 130: 0.8975
Accuracy at step 140: 0.8943
Accuracy at step 150: 0.8995
Accuracy at step 160: 0.8986
Accuracy at step 170: 0.9007
Accuracy at step 180: 0.9052
Accuracy at step 190: 0.898
Adding run metadata for 199
Accuracy at step 200: 0.9035
Accuracy at step 210: 0.9061
Accuracy at step 220: 0.9047
Accuracy at step 230: 0.8998
Accuracy at step 240: 0.9078
Accuracy at step 250: 0.8985
Accuracy at step 260: 0.9082
Accuracy at step 270: 0.9088
Accuracy at step 280: 0.9051
Accuracy at step 290: 0.9053
Adding run metadata for 299
Accuracy at step 300: 0.9088
Accuracy at step 310: 0.9061
Acc

In [7]:
sess.close()

In [8]:
print('tensorboard --logdir=' + log_dir)

tensorboard --logdir=/home/jovyan/work/logs/alex/initial
